# Evaluation

In [522]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [550]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments, evaluate
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [577]:
NUM_VIDEOS = 200
GRID_SIZE = 2
BINS = [int(180/10), int(256/10)]
# negative value is average; -2 averages two frames, takes every 2nd frame (only skips one) (if frame_id % 2 == 0).
HIST_FRAME_SKIP = 5
REFRESH = True

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [578]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE,
                                      BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 200 - Grid 2 - Bins [18, 25] - Skip 5


In [579]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = prep.get_test_video_set(NUM_VIDEOS, GRID_SIZE, BINS, n=100)

processing 100/100

In [568]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:    {:d}".format( len(training_set)) )
print("Num. segments:  {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:       {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:     {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms: {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

print("\nTEST SET:")
print("Size: {:d}".format( len(test_set) ))

TRAINING SET:
Num. videos:    20
Num. segments:  2198
Duration:       7,556.4 s
Num frames:     212462
Num histograms: 43361

TEST SET:
Size: 10


# Small manual test

In [583]:
for i, test_segment in enumerate(test_set):
    found = search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_CHISQR_ALT,
                                    5, HIST_FRAME_SKIP, [0,1])
    print('Found {} - Expected {}'.format(found, labels[i]))
    if i == 1:
        break


print()
for i, test_segment in enumerate(test_set):
    found = search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_CHISQR_ALT,
                                    5, HIST_FRAME_SKIP, [1])
    
    print('Found {} - Expected {}'.format(found, labels[i]))
    if i == 1:
        break
        
print()
for i, test_segment in enumerate(test_set):
    found = search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_CHISQR_ALT,
                                    5, HIST_FRAME_SKIP, [0])
    
    print('Found {} - Expected {}'.format(found, labels[i]))
    if i == 1:
        break

Found ('00123.mp4', 6154.0) - Expected ('00123.mp4', 5435, 5914)
Found ('00143.mp4', 5973.5) - Expected ('00143.mp4', 5539, 6018)
Found ('00086.mp4', 7728.0) - Expected ('00086.mp4', 6865, 7365)
Found ('00002.mp4', 3625.5) - Expected ('00002.mp4', 3225, 3824)
Found ('00193.mp4', 10481.5) - Expected ('00193.mp4', 7871, 8371)
Found ('00170.mp4', 1541.0) - Expected ('00170.mp4', 1270, 1869)
Found ('00149.mp4', 3654.0) - Expected ('00149.mp4', 3341, 3821)
Found ('00169.mp4', 26947.0) - Expected ('00169.mp4', 25865, 26365)
Found ('00158.mp4', 15246.0) - Expected ('00158.mp4', 14338, 14838)
Found ('00149.mp4', 3654.0) - Expected ('00149.mp4', 3553, 4033)
Found ('00112.mp4', 1421.5) - Expected ('00112.mp4', 1062, 1541)
Found ('00192.mp4', 34.0) - Expected ('00192.mp4', 73, 673)
Found ('00132.mp4', 1035.0) - Expected ('00132.mp4', 736, 1216)
Found ('00160.mp4', 1354.0) - Expected ('00160.mp4', 1028, 1508)
Found ('00064.mp4', 1519.0) - Expected ('00064.mp4', 1191, 1670)
Found ('00105.mp4', 3922

KeyboardInterrupt: 

## Run model on test set

In [572]:
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP)
    
print()
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP, [0])   

print()
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP, [1])   

312 ms ± 8.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
305 ms ± 6.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
304 ms ± 4.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
303 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
303 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
304 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

193 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
192 ms ± 2.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
193 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
198 ms ± 6.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
192 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
195 ms ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

196 ms ± 2.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
218 ms ± 19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
198 ms ± 1.82 ms per loop (mean ± std. dev. of 7 r

In [ ]:
%%time
results = []

for i, test_segment in enumerate(test_set):
    print("\rSearching segment {}/{}".format(i+1, len(test_set), len(test_segment)), end='', flush=True)
    
    results.append(search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_CHISQR_ALT, 5, 
                                    HIST_FRAME_SKIP, [0]))


Searching segment 20/100

## Evaluate performance

In [497]:
movie_results, start_frame_dist = evaluate(results, labels)

fractions = (movie_results[0] / movie_results[2]*100 if movie_results[2] > 0 else 0, \
             movie_results[1] / movie_results[0]*100 if movie_results[0] > 0 else 0)

print("TEST RESULTS\n")

printParams()
print("\nCorrect video: {:d} / {:d} ({:.1f}%)".format(movie_results[0], movie_results[2], fractions[0]))
print("Inside fragment: {:d} / {:d} ({:.1f}%)".format(movie_results[1], movie_results[0], fractions[1]))
print("Average distance to center of segment: {:.0f} +/- {:.0f} frames (approx. {:.1f} sec)".format(
    start_frame_dist[0], start_frame_dist[1], start_frame_dist[0]/30))

TEST RESULTS

Num. Vid 2 - Grid 2 - Bins [45, 64] - Skip 2

Correct video: 10 / 10 (100.0%)
Inside fragment: 10 / 10 (100.0%)
Average distance to center of segment: 80 +/- 54 frames (approx. 2.7 sec)


In [582]:
%%time

for i in range(200):
    print(i, end='\r')

CPU times: user 5.04 ms, sys: 37 µs, total: 5.08 ms
Wall time: 3.26 ms
